<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Attention_Mechanisms_and_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Transformer

class PositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, hidden_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / hidden_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(1)  # Add batch dimension

    def forward(self, x):
        return x + self.pe[:x.size(0), :].to(x.device)

class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_heads, num_layers, max_len=5000):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim, max_len)
        self.transformer = Transformer(d_model=hidden_dim, nhead=num_heads, num_encoder_layers=num_layers, num_decoder_layers=num_layers, batch_first=False)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Add embeddings and positional encoding
        src_emb = self.positional_encoding(self.embedding(src))
        tgt_emb = self.positional_encoding(self.embedding(tgt))

        # Pass through Transformer
        output = self.transformer(src_emb, tgt_emb, src_mask=src_mask, tgt_mask=tgt_mask)
        return self.fc_out(output)

# Example hyperparameters
input_dim = 5000  # Vocabulary size
output_dim = 5000  # Vocabulary size
hidden_dim = 512  # Hidden layer dimension
num_heads = 8  # Number of attention heads
num_layers = 6  # Number of Transformer layers

# Initialize model, optimizer, and example data
model = TransformerModel(input_dim, output_dim, hidden_dim, num_heads, num_layers)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Example data (source and target sequences)
src = torch.randint(0, input_dim, (10, 32))  # 10 words, 32 batch size
tgt = torch.randint(0, output_dim, (20, 32))  # 20 words, 32 batch size

# Forward pass
output = model(src, tgt)

print("Output shape:", output.shape)